### At the top press File > Save copy in Drive > a copy will open > (in the new copy tab) press Runtime > press Run All > answer questions > graph and table of predicted weights will appear


video guide: https://www.youtube.com/watch?v=qsymcjY2x0E

### My General Weight Loss Tips:


1.   2 of the most effective Dietary restrictions
  *   Cut out processed sugar (careful for sugar withdrawal, get natural sugars from fruit)
  *   Limit intake of fried foods
2.   Get the right amount of sleep (https://www.sleepfoundation.org/press-release/national-sleep-foundation-recommends-new-sleep-times)
3.   Exercise at least 20 minutes on average daily (start off slow if you're new to exercise)
4.   DON'T weigh yourself everyday.  I suggest a max of twice a week.  Preferably once every week or even once every 2 weeks.
5. We recommend starting with a diet that cuts 20% of what is needed to maintain your weight
6. Avoid Fad and Extreme Diets (less than 50% of caloric expenditure)


### Math and Science


Credits...

Special thanks to https://www.youtube.com/watch?v=F2pAu9PzgEE (Juan Kooper) for help with interactive plotting

and bmi-calories.com for the mathematical formulas


Formula: $$\frac{weight (kg)}{height (m)^2}$$

The formula for BMI is weight in kilograms divided by height in meters squared. If height has been measured in centimeters, divide by 100 to convert this to meters.

Equations taken from https://bmi-calories.com/calorie-intake-calculator.html

The Original Harris-Benedict Equation (The default equation for this program)

Men ce = 66.4730 + (13.7516 x weight in kg) + (5.0033 x height in cm) – (6.7550 x age in years)

Women ce = 655.0955 + (9.5634 x weight in kg) + (1.8496 x height in cm) – (4.6756 x age in years)

The Mifflin St Jeor Equation

Men ce = (10 x weight in kg) + (6.25 x height in cm) - (5 x age in years) + 5 (measured in Kcal/day)

Women ce = (10 x weight in kg) + (6.25 x height in cm) - (5 x age in years) - 161 (measured in Kcal/day)

Additional Sources:
https://www.ncbi.nlm.nih.gov/pubmed/18688113

https://www.ncbi.nlm.nih.gov/pubmed/17324656

https://www.researchgate.net/post/which_formula_are_recommended_by_nutritionists_and_scientists_to_measure_BASAL_METABOLIC_RATE

weight change (pounds) = caloric intake - caloric expenditure / 3500 (calories)


### Code

In [15]:
# TODO
# Maybe add sliders later
# But I don't think it's that good because people will try to go on more extreme diets and unrealistic activity levels

In [16]:
from ipywidgets import interact, interactive
import ipywidgets as widgets
import IPython
from IPython.display import display
from IPython.display import clear_output
import datetime

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 200)
import numpy as np
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected = True)
import seaborn as sns
import plotly.graph_objects as go
from IPython.display import Javascript

In [17]:
def configure_plotly_browser_state():
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
            requirejs.config({
                paths: {
                    base: '/static/base;',
                    plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
                },
            });
        </script>
        '''))

In [18]:
# UI Elements, entry page
fields = ['weight','height','age']
input_fields = [widgets.FloatText(description = field, ) for field in fields]
activity_level_field = widgets.FloatText(description = 'activity level (hours exercise per week)', style=dict(description_width='initial'))
diet_level_field = widgets.FloatText(description = '% calories you will cut (0-50)%)', style=dict(description_width='initial'))
date_field = widgets.Text(placeholder = 'date as YYYY-MM-DD (leave blank for today)', style=dict(description_width='initial'))

weight_units_dropdown = widgets.Dropdown(
    options=['pounds','kilograms'],
    value='pounds',
    description='Weight Unit')

height_units_dropdown = widgets.Dropdown(
    options=['inches','centimeters'],
    value='inches',
    description='Height Unit')

gender_dropdown = widgets.Dropdown(
    options=['male','female'],
    value='male',
    description= 'Gender')

submit_button = widgets.Button(description='Submit')

back_button = widgets.Button(description='Back')

# format box_scene#_column#
box_01_01 = widgets.VBox([input_fields[0], input_fields[1], input_fields[2], activity_level_field, diet_level_field, submit_button])
box_01_02 = widgets.VBox([weight_units_dropdown, height_units_dropdown, gender_dropdown, date_field])


# FUTURE SLIDER CODE
# diet_level_slider = widgets.IntSlider(value = 50, min = 50, max = 150, step = 1, Description = 'Change Activity Level (hours)')
# activity_level_slider = widgets.IntSlider(value = 0, min = 0, max = 24, step = 1, Description = 'Change Activity Level (hours)')
# box_02_01 = widgets.VBox([diet_level_slider, activity_level_slider,back_button])

In [19]:
class Weight_Loss_Projection():
    
    def __init__(self):
        self.weight_pounds = None
        self.weight_kilograms = None
        self.height = None
        self.gender = None
        self.age = None
        self.date = None
        self.activity_level = None
        self.diet_level = None
        self.user_table = None
        self.user_df = None

    def update_on_submit(self):
        if weight_units_dropdown.value == 'pounds':
            self.weight_pounds = input_fields[0].value
            self.weight_kilograms = self.weight_pounds * 0.453592
        else:
            self.weight_kilograms = input_fields[0].value
            self.weight_pounds = self.weight_kilograms / 0.453592
        # height is easier because it isn't changed, but maybe need to implement later for growing kids   
        self.height = input_fields[1].value
        if height_units_dropdown.value == 'inches':
            self.height *= 2.54
        self.gender = gender_dropdown.value
        self.age = input_fields[2].value
        if date_field.value == '' or date_field.value == None :
            self.date = datetime.date.today()
        else:
            self.date = datetime.datetime.strptime(date_field.value, '%Y-%m-%d')
        self.activity_level = activity_level_field.value
        self.diet_level = diet_level_field.value
        
        # FUTURE SLIDER CODE
        # diet_level_slider.value = diet_level_field.value
        # activity_level_slider.value =  activity_level_field.value
        self.diet_level = diet_level_field.value
        self.user_table = []
        self.user_df = None
    
    def update_sliders(self, b):
        clear_output()
        self.activity_level = activity_level_slider.value
        self.diet_level = diet_level_slider.value
        self.show_graph()
        
    def ce_bmi_calculator(self):

      # bmi calculation, height * 100 because was in cm
        bmi = self.weight_kilograms / (self.height / 100) ** 2

        if self.gender == 'male':
            # ce = 10 * self.weight_kilograms + 6.25 * self.height - 5 * self.age + 5
            ce = 66.4730 + 13.7516 * self.weight_kilograms + 5.0033 * self.height - 6.7550 * self.age
        else:
            # ce = 10 * self.weight_kilograms + 6.25 * self.height - 5 * self.age - 161
            ce = 655.0955 + 9.5634 * self.weight_kilograms + 1.8496 * self.height - 4.6756 * self.age
        # activity level correction
        ce *= (1.2 + 0.075 * activity_level_field.value)
        return ce, bmi
    
    def show_graph(self):
        clear_output()
        self.update_on_submit()
        user_table = []

        # ce = caloric expenditure, bmi = body mass index
        ce, bmi = self.ce_bmi_calculator()
        if weight_units_dropdown.value == 'pounds':
            user_table.append((self.date, self.weight_pounds, bmi, ce, 0))
        else:
            user_table.append((self.date, self.weight_kilograms, bmi, ce, 0))
        self.calculate_weight_steps(ce)

        if weight_units_dropdown.value == 'pounds':
            temp_weight_title = ('Weight (lbs.)')
        else:
            temp_weight_title = ('Weight (kg.)')

        # 2nd scene UI layout
        box_02_01 = widgets.VBox([back_button])
        # display(widgets.HBox([box_02_01]))
            
        # plotting
        configure_plotly_browser_state()
        init_notebook_mode(connected=False)
        trace0 = go.Scatter(x = self.user_df['Date'], y = self.user_df.round(1)['Weight'], mode = 'markers', marker_size = 4)
        trace1 = go.Scatter(x = self.user_df['Date'], y = self.user_df['Calories In'], mode = 'markers', marker_size = 4)
        data = [trace0]
        data2 = [trace1]
        layout = {"title": 'Weight Loss Projection | Hover with Cursor - format is (Date, Weight)',
                  "xaxis":{"title":"Date",
                           "zeroline":False},
                  "yaxis":{"title":temp_weight_title,
                           "zeroline":False},
                  "width": 900,
                  "height": 800}
        layout2 = {"title": 'Weight Loss Projection | Hover with Cursor - format is (Date, Calories In)',
                  "xaxis":{"title":"Date",
                           "zeroline":False},
                  "yaxis":{"title":'Calories In',
                           "zeroline":False},
                  "width": 900,
                  "height": 800}
        iplot({'data':data,
               "layout":layout})
        iplot({'data':data2,
               "layout":layout2})
        display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 20000})'''))
        print("Table of Daily Weight, BMI, Calories In, and Calories Out")
        display(self.user_df)

    def calculate_weight_steps(self, ce):
        for i in range(180):
            calorie_deficit = ce - ce * (1-self.diet_level / 100)
            self.weight_pounds -= calorie_deficit / 3500
            self.weight_kilograms -= calorie_deficit / 3500 * 0.453592
            self.date += datetime.timedelta(days=1)
            # ce = caloric expenditure, bmi = body mass index
            ce, bmi = self.ce_bmi_calculator()
            if weight_units_dropdown.value == 'pounds':
                self.user_table.append((self.date, self.weight_pounds, bmi, ce, ce - calorie_deficit))
            else:
                self.user_table.append((self.date, self.weight_kilograms, bmi, ce, calorie_deficit))
        self.user_df = pd.DataFrame(data = self.user_table,
                                    columns = ['Date','Weight','BMI','Calories Out','Calories In'])
    # b for button, necessary to use on_click from ipywidgets    
    def scene_01(self, b):
        display(widgets.HBox([box_01_01, box_01_02]))
        submit_button.on_click(self.scene_02)
        
    def scene_02(self, b):
        self.show_graph()

        # FUTURE SLIDER CODE
        # activity_level_slider.observe(self.update_sliders, names='value')
        # diet_level_slider.observe(self.update_sliders, names='value')

        back_button.on_click(self.scene_01)




In [20]:
# note at day 120, my bmi is <30 so it switches formulas, causing the calorie descrepancy
def main():
    """
    The program will ask for your daily metabolic rate as calculated by the link above
    You will be able to adjust your desired intensity level (% you cut from daily metabolic rate)
    as well as your desired timespan
    """
    my_projection = Weight_Loss_Projection()
    display(widgets.HBox([box_01_01, box_01_02]))
    submit_button.on_click(my_projection.scene_02)

## UI

In [22]:
# Click the play button to the left to put in new numbers
# Note 1: Graph may take a while to load at first (because of python dependancy installations)
# Note 2: you can enter negative numbers for the "% calories you will cut" if you're trying to gain weight
main()